In [1]:
from scipy.ndimage import gaussian_filter
import numpy  as np
import matplotlib.pyplot as plt
import time
from utils import *
import copy
from hcipy import *
import os.path
import csv
from astropy.io import fits
from processing import *
%matplotlib inline

# Import Files

## Input File Names

In [2]:
## Input the FITS name and the camera name of as many FITS file as you want

saved_file_name = "Five_Shutter_Analysis" # Name you want to use to save images

fits_name1 = "ProEM_HS_512BX3.fits" # Type name of FITS file you want to read
camera_name1 = "ProEM" # Name of the first shutter. Used for legends

fits_name2 = "ORCA Quest.fits" # Name of Second Shutter Data. Is not used if second_shutter_test = False
camera_name2 = "ORCA Quest" # Name of Second Shutter. Used for Legends

fits_name3 = "Kinetix.fits" # Name of Third Shutter Data. Is not used if second_shutter_test = False
camera_name3 = "Kinetix" # Name of Third Shutter. Used for Legends

fits_name4 = "iXon_897.fits" # Name of Fourth Shutter Data. Is not used if second_shutter_test = False
camera_name4 = "iXon 897" # Name of Fourth Shutter. Used for Legends

fits_name5 = "Prime_BSI.fits" # Name of Fifth Shutter Data. Is not used if second_shutter_test = False
camera_name5 = "Prime BSI" # Name of Fifth Shutter. Used for Legends

sigma = 5 # determines contrast level, e.g. sigma=5 --> 5-sigma contrast curve

## Load Fits Files

In [3]:
fits_file1 = fits.open(fits_name1)
fits_file2 = fits.open(fits_name2)
fits_file3 = fits.open(fits_name3)
fits_file4 = fits.open(fits_name4)
fits_file5 = fits.open(fits_name5)
fits_files = [fits_file1, fits_file2, fits_file3, fits_file4, fits_file5]
camera_names = [camera_name1, camera_name2, camera_name3, camera_name4, camera_name5]

## Load Header Data

In [4]:
wavelengths = []
pupil_diameters = []
mags = []
qs = []
shutters = []
fpss = []
read_noises = []
dark_currents = []
seeings = []
outer_scales = []
velocitys = []
quantum_efficiencys = []
exposure_times = []
exposure_numbers = []
for i in fits_files:
    wavelength = i[0].header['Wavelnth']
    wavelength = wavelength.split(' ')
    wavelengths.append(float(wavelength[0]))
    pupil_diameter = i[0].header['PUPDIAMT']
    pupil_diameter = pupil_diameter.split(' ')
    pupil_diameters.append(float(pupil_diameter[0]))
    mag = i[0].header['PRIMAG']
    mag = mag.split(' ')
    mags.append(float(mag[0]))
    q = i[0].header['Q']
    q = q.split(' ')
    qs.append(float(q[0]))
    shutter = i[0].header['NSubdivi']
    shutter= shutter.split(' ')
    shutter=float(shutter[0])
    if shutter == 1:
        shutters.append("Global")
    else:
        shutters.append("Rolling")
    fps = i[0].header['FPS']
    fps = fps.split(' ')
    fpss.append(fps[0])
    read_noise = i[0].header['RdNoise']
    read_noise = read_noise.split(" ")
    read_noises.append(read_noise[0])
    dark_current = i[0].header['DarkCurr']
    dark_current = dark_current.split(" ")
    dark_currents.append(dark_current[0])
    quantum_efficiency = i[0].header['QE']
    quantum_efficiency = quantum_efficiency.split(" ")
    quantum_efficiencys.append(quantum_efficiency[0])
    seeing = i[0].header['Seeing']
    seeing = seeing.split(" ")
    seeings.append(seeing[0])
    outer_scale = i[0].header['OutScale']
    outer_scale = outer_scale.split(' ')
    outer_scales.append(outer_scale[0])
    velocity = i[0].header['Velocity']
    velocity = velocity.split(' ')
    velocitys.append(velocity[0])
    exposure_time = i[0].header['ExpoTime']
    exposure_time = exposure_time.split(" ")
    exposure_times.append(exposure_time[0])
    exposure_number = i[0].header['NumExpos']
    exposure_number = exposure_number.split(" ")
    exposure_numbers.append(exposure_number[0])

# Combine Image Data

In [6]:
ims_out_as = []
for i in fits_files:
    print("Combining Image Data for ", i[0].header['Title'])
    npix = int(np.sqrt(np.prod(i[1].data.shape)))
    ims_out = []
    for j in range(1, len(i)):
        im = i[j].data
        im_out = im.copy().reshape([npix,npix])
        ims_out.append(np.array(im_out))
    ims_out_as.append(np.array(ims_out))
print("Image Combining Complete")

Combining Image Data for  ProEM_HS_512BX3
Combining Image Data for  ORCA Quest
Combining Image Data for  Kinetix
Combining Image Data for  iXon_897
Combining Image Data for  Prime_BSI
Image Combining Complete


# Image Processing

In [ ]:
## Image Processing
# includes preprocessing, taking FTs, power spectra, and ACFs

# Function Parameters - see processing.py for more detail
# ims           - input image array
# ims_ft        - input FT array
# gsigma        - std deviation for the Gaussian kernel
# subframe_size - final image size in pixels
# HWHM          - half-wavelength at half maximum for supergaussian window
# m             - order of supergaussian window
# scaling       - determines radial cutoff (fcut) for PS
ims_ps=[]
FTs =[]
PSs = []
Av_PSs = []
ACFs = []
for i in range(len(fits_files)):
    print("Processing ", fits_files[i][0].header['Title'] )
    print("Performing Image Preprocessing")
    ims_p = image_preprocessing(ims_out_as[i], 10, 550)
    #              parameters: (ims, gsigma, subframe_size)
    ims_ps.append(ims_p)
    print("Taking Fourier Transform")
    FT = fourier_transform(ims_p, 100, 4)
    #              parameters: (ims, HWHM, m)
    FTs.append(FT)
    print("Taking Power Spectrum")
    PS, Av_PS = power_spectrum(FT, qs[i], wavelengths[i], pupil_diameters[i], 1.)
    #              parameters: (ims_ft, wavelength, pupil_diameter, scaling, HWHM, m)
    PSs.append(PS)
    Av_PSs.append(Av_PS)
    print("Performing Auto Correlation Function")
    ACF = generate_ACF(Av_PS)
    #              parameters: (ims_ps)
    ACFs.append(ACF)
    print()
print("Processing Complete")

Processing  ProEM_HS_512BX3
Performing Image Preprocessing
Taking Fourier Transform
Taking Power Spectrum


# Generate Speckle Contrast Curve

In [ ]:
ACF_ccs = []
ACF_xaxs = []

for i in range(len(fits_files)):
    plate_scale = wavelengths[i] / (pupil_diameters[i] * qs[i]) * 206265. # (arcsec/pixel)
    rad_ACF = radial_data(np.abs(ACFs[i]), annulus_width=2)
    ACF_cc = -2.5 * np.log10((1. - np.sqrt(1. - (2 * (sigma * rad_ACF.std)) ** 2)) / (2 * (sigma * rad_ACF.std)))
    ACF_ccs.append(ACF_cc)
    ACF_xax = np.array(range(len(rad_ACF.mean))) * plate_scale # arcsec 
    ACF_xaxs.append(ACF_xax) 
    

f = plt.figure(figsize=(15,12))
for i in range(len(fits_files)):
    label = fits_files[i][0].header['Title'] + ", FPS = "+ fpss[i] + ", Shutter = " + shutters[i] + ", Read Noise = " + read_noises[i] + ", Dark Current = " + dark_currents[i] + ", QE = " + quantum_efficiencys[i]
    plt.plot(ACF_xaxs[i], ACF_ccs[i], label = label , lw=3)
title = "\nPrimary Magnitude = " + str(mags[0]) +  ", Exposure Time = " + exposure_times[0] + ", Number of Exposures = " + exposure_numbers[0] + "\nSeeing = " + seeings[0] + ", Outer Scale = " + outer_scales[0] + ", Wind Velocity = " + velocitys[0]
plt.xlim(0.0, 2.0)
plt.gca().invert_yaxis()
plt.legend(loc='lower left')
plt.ylabel(r'' + str(sigma) + ' $\sigma$ Contrast (mag)')
plt.xlabel(r'Separation (arcsec)')
plt.title('VIPER Conventional Speckle' + title)
plt.savefig(saved_file_name + ".jpg", bbox_inches = 'tight',  dpi = 300)